# Lecture 7: Working with Claude API and Reasoning Models

### Overview

Let's finally dive into the nitty-gritty and see how we can code with Claude.

This notebook explores the Anthropic Claude API, focusing on how to leverage both standard chat capabilities and advanced reasoning features. We'll examine how to structure effective system prompts and create applications that utilize the model's extended thinking capabilities.

### Objectives
- Set up and configure the Anthropic Claude API
- Create tailored system prompts for specific use cases
- Compare standard responses with reasoning-enhanced outputs
- Analyze the "thinking" process in reasoning models

### Challenge
Take the code we have used in this lecture and see if you can beat my overall score. Let's turn this into a friendly competition for all those tackling this project and compare the results you get :D

Place your submission in the community-contributions folder by creating a folder with your name in it.

### Version History

Below you will find any updates I have made to the notebooks.

**Please note** that some of the code may be different to what you have seen in the lectures as there may be bug fixes or improvements but the general objective of the notebook will remain the same.

| Date | Version | Description of Changes |
|------|---------|------------------------|
| 2025-03-01 | v1.0 | Initial version |



#### Step 1: Import libraries and load the environment variables

In [1]:
import os
import anthropic
from dotenv import load_dotenv
from IPython.display import Markdown, display

load_dotenv()

ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

if ANTHROPIC_API_KEY is None:
    raise Exception("API key is missing")

#### Step 2: Call the API

In [2]:
client = anthropic.Anthropic()

message = client.messages.create(
    model="claude-3-7-sonnet-20250219",
    max_tokens=1000,
    temperature=1,
    system="You are a world-class poet. Respond only with short poems.",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Why is the ocean salty?"
                }
            ]
        }
    ]
)
print(message.content[0].text)

# Salt Sea's Tale

From ancient rocks, minerals flow,
Through rivers' journey, down they go.
Eons of weeping from the land,
While water stays, salt makes its stand.

The ocean weeps not, but collects—
Earth's seasoned tears, time's dialect.


#### Step 3: Customize the Prompt

Customer service representative

In [3]:
complaint = "You have over charged me for this product. I demand a refund."

In [4]:
system_message = "You are an expert customer service representative. Respond realisticly."

prompt = f"""
Write a brief professional email response to an unhappy customer with the following issue:

{complaint}

Requirements:
- Keep under 150 words
- Maintain a professional but empathetic tone
- Acknowledge the issue clearly
- Offer a specific solution
- Include a follow-up action
"""

message = client.messages.create(
    model="claude-3-7-sonnet-20250219",
    max_tokens=1000,
    temperature=0.7,
    system=system_message,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt
                }
            ]
        }
    ]
)

chat_response = message.content[0].text
display(Markdown(chat_response))

Subject: Regarding Your Refund Request - We're Here to Help

Dear Valued Customer,

Thank you for bringing this overcharging issue to our attention. I sincerely apologize for this error and understand your frustration.

I've reviewed your account and can confirm that an incorrect amount was charged. We take billing accuracy very seriously, and I'm processing a full refund for the overcharged amount immediately. You should see this reflected in your account within 3-5 business days.

As a gesture of goodwill for the inconvenience caused, we're also adding a 10% discount code for your next purchase: THANKYOU10.

If you don't see the refund by Friday, or have any other questions, please contact me directly at customercare@company.com or call 555-123-4567.

Again, I apologize for this mistake and appreciate your patience.

Best regards,
Customer Service Representative

#### Step 4: Rate the response using a reasoning model

In [5]:
system_message = "You are an expert customer service representative. Evaluate the email response to be sent to a customer complaint."

reasoning_prompt = f"""
A customer has made a complaint.

{complaint}

Evaluate this customer service email response.

{chat_response}

Rate each category with ONE LINE explanation:

1. Professionalism (1-10): [Score] - [One-line explanation]
2. Empathy (1-10): [Score] - [One-line explanation]
3. Problem Resolution (1-10): [Score] - [One-line explanation]
4. Clarity (1-10): [Score] - [One-line explanation]
5. Brand Representation (1-10): [Score] - [One-line explanation]

TOTAL SCORE: [X/100]
"""

message = client.messages.create(
    model="claude-3-7-sonnet-20250219",
    max_tokens=5000,
    thinking={
        "type": "enabled",
        "budget_tokens": 3000
    },
    temperature=1,
    system=system_message,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": reasoning_prompt
                }
            ]
        }
    ]
)
reasoning_response = message.content
print(reasoning_response)

[ThinkingBlock(signature='ErUBCkYIAxgCIkA4kpqyS0TVKFhDaVDnLJWDE3JfpcroeHfNboIumF0Gslg79h0+QidgyZHoE1hmuLvuAY783QQZHeboHExb7P3+EgwHEvSosNNsph7wBFEaDAihYK897uqwMaZ2GSIw3wxtd5Y/+hq/Tyk8KIVn41loKlHlfKaUhpEV65n9IahyKj/oorLsC5Z6qxqRZ3TwKh1caIFmy8ThEMl1ZjGARlFVLxZLng9OSUO8vDOpWhgC', thinking='Let me evaluate this customer service email response based on the five provided categories:\n\n1. Professionalism (1-10):\nThe email maintains a professional tone throughout, with proper salutation and closing. The representative acknowledges the issue clearly, offers a formal apology, and provides specific contact information for follow-up. The language is respectful and business-appropriate. There are no grammatical errors or informal language. The response is thorough without being overly lengthy. I would rate this 9/10 - Highly professional with appropriate formal tone and structure.\n\n2. Empathy (1-10):\nThe response shows empathy by acknowledging the customer\'s frustration, apologizing sincerely,

In [6]:
display(Markdown(reasoning_response[1].text))

# Customer Service Email Evaluation

## Detailed Assessment

1. Professionalism (1-10): 9 - Maintains formal tone with appropriate business language, proper structure, and error-free writing.

2. Empathy (1-10): 8 - Acknowledges frustration, offers sincere apology, and demonstrates understanding of the impact through both words and compensation.

3. Problem Resolution (1-10): 10 - Excellent resolution with immediate action, verification of the error, clear timeline, and multiple follow-up options.

4. Clarity (1-10): 9 - Communicates information in a straightforward manner with logical structure and specific details about the resolution process.

5. Brand Representation (1-10): 9 - Represents the company as accountable, customer-focused, and willing to make things right through both words and actions.

## TOTAL SCORE: 90/100

This is an excellent customer service response that addresses the complaint directly, takes immediate action, and goes beyond mere problem-solving by offering additional compensation. The response is professional yet personable, and provides clear next steps with specific timeline expectations.